# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
import sys
sys.version

'3.6.3 (default, Nov 13 2017, 19:33:03) \n[GCC 4.8.5 20150623 (Red Hat 4.8.5-4)]'

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [3]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [4]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    source_id_text = [[source_vocab_to_int[word] for word in sentence.split()] for sentence in source_text.split('\n')]
    target_id_text = [[target_vocab_to_int[word] for word in sentence.split()] + [target_vocab_to_int['<EOS>']] for sentence in target_text.split('\n')]
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.4.0
Default GPU Device: /device:GPU:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [8]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    _input = tf.placeholder(dtype=tf.int32, shape=(None, None), name='input')
    _targets = tf.placeholder(dtype=tf.int32, shape=(None, None), name='target')
    _lr = tf.placeholder(dtype=tf.float32, name='learning_rate')
    _keep_probability = tf.placeholder(dtype=tf.float32, name='keep_prob')
    _target_seq_length = tf.placeholder(dtype=tf.int32, shape=(None,), name='target_sequence_length')
    _max_target_seq_length = tf.reduce_max(_target_seq_length, name='max_target_len')
    _source_seq_length = tf.placeholder(dtype=tf.int32, shape=(None,), name='source_sequence_length')
    return _input, _targets, _lr, _keep_probability, _target_seq_length, _max_target_seq_length, _source_seq_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [9]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    ending = tf.strided_slice(target_data, [0,0], [batch_size, -1], [1,1])
    dec_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)
    return dec_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [10]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    enc_embed_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    
    # RNN cell
    def make_cell(rnn_size):
        enc_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=1))
        enc_cell_with_dropout = tf.contrib.rnn.DropoutWrapper(enc_cell, output_keep_prob=keep_prob)
        return enc_cell_with_dropout
    
    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    enc_output, enc_state = tf.nn.dynamic_rnn(enc_cell, enc_embed_input, sequence_length=source_sequence_length, dtype=tf.float32)
    return enc_output, enc_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input, 
                                                            sequence_length=target_sequence_length,
                                                            time_major=False)
    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, training_helper, encoder_state, output_layer)
        
    training_decoder_output = tf.contrib.seq2seq.dynamic_decode(training_decoder, impute_finished=True, maximum_iterations=max_summary_length)[0]
    
    return training_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [12]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    # print("======in function:========")
    # print("encoder_state: {}".format(len(encoder_state)))
    # print("dec_embedding: {}".format(dec_embeddings))
    # print("start_of_sequence_id: {}".format(start_of_sequence_id))
    # print("end_of_sequence_id: {}".format(end_of_sequence_id))
    # print("max_target_sequence_length: {}".format(max_target_sequence_length.shape))
    # print("vocab_size: {}".format(vocab_size))
    # print("output_layer: {}".format(output_layer))
    # print("batch_size: {}".format(batch_size))
    # print("keep_prob: {}".format(keep_prob.shape))
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='start_tokens')
        
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, start_tokens, end_of_sequence_id)
        
    inference_decode = tf.contrib.seq2seq.BasicDecoder(dec_cell, inference_helper, encoder_state, output_layer)
        
    inference_decoder_output = tf.contrib.seq2seq.dynamic_decode(inference_decode, 
                                                                     impute_finished=True, 
                                                                     maximum_iterations=max_target_sequence_length)[0]
    return inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [13]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    # 1. Decoder Embedding
    dec_embedings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embedings, dec_input)
    
    # 2. construct the decoder cell
    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=1))
        dec_cell_with_dropout = tf.contrib.rnn.DropoutWrapper(dec_cell, output_keep_prob=keep_prob)
        return dec_cell_with_dropout

    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    # 3. Dense layer to translate the decoder's output at each time 
    # step into a choice from the target vocabulary
    output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    # 4. Set up a training decoder and an inference decoder
    with tf.variable_scope("decode"):
        training_decoder_output = decoding_layer_train(encoder_state, 
                                                       dec_cell, 
                                                       dec_embed_input, 
                                                       target_sequence_length, 
                                                       max_target_sequence_length, 
                                                       output_layer, 
                                                       keep_prob)
        
    with tf.variable_scope("decode", reuse = True):
        inference_decode_output = decoding_layer_infer(encoder_state, 
                                                       dec_cell, 
                                                       dec_embedings, 
                                                       target_vocab_to_int['<GO>'],
                                                       target_vocab_to_int['<EOS>'], 
                                                       max_target_sequence_length,
                                                       target_vocab_size, 
                                                       output_layer, 
                                                       batch_size, 
                                                       keep_prob)
    return training_decoder_output, inference_decode_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [14]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    
    _, enc_state = encoding_layer(input_data, rnn_size, num_layers, keep_prob,
                                  source_sequence_length, source_vocab_size, enc_embedding_size)
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    training_decoder_output, inference_decoder_output = decoding_layer(dec_input,enc_state,target_sequence_length, 
                                                                       max_target_sentence_length,rnn_size,num_layers,
                                                                       target_vocab_to_int,target_vocab_size,batch_size,
                                                                       keep_prob, dec_embedding_size) 
    return training_decoder_output, inference_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [15]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 512
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 200
decoding_embedding_size = 200
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.5
display_step = 2

### Build the Graph
Build the graph using the neural network you implemented.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    2/538 - Train Accuracy: 0.2863, Validation Accuracy: 0.3471, Loss: 4.3121
Epoch   0 Batch    4/538 - Train Accuracy: 0.2955, Validation Accuracy: 0.3583, Loss: 3.9442
Epoch   0 Batch    6/538 - Train Accuracy: 0.3452, Validation Accuracy: 0.3864, Loss: 3.5753
Epoch   0 Batch    8/538 - Train Accuracy: 0.3223, Validation Accuracy: 0.3869, Loss: 3.4727
Epoch   0 Batch   10/538 - Train Accuracy: 0.3352, Validation Accuracy: 0.4119, Loss: 3.3778
Epoch   0 Batch   12/538 - Train Accuracy: 0.3676, Validation Accuracy: 0.4348, Loss: 3.2307
Epoch   0 Batch   14/538 - Train Accuracy: 0.3715, Validation Accuracy: 0.4339, Loss: 3.0388
Epoch   0 Batch   16/538 - Train Accuracy: 0.4066, Validation Accuracy: 0.4462, Loss: 2.8349
Epoch   0 Batch   18/538 - Train Accuracy: 0.3982, Validation Accuracy: 0.4625, Loss: 2.9034
Epoch   0 Batch   20/538 - Train Accuracy: 0.4366, Validation Accuracy: 0.4709, Loss: 2.6669
Epoch   0 Batch   22/538 - Train Accuracy: 0.4367, Validation Accuracy

Epoch   0 Batch  180/538 - Train Accuracy: 0.6248, Validation Accuracy: 0.5978, Loss: 0.8115
Epoch   0 Batch  182/538 - Train Accuracy: 0.5672, Validation Accuracy: 0.6010, Loss: 0.8533
Epoch   0 Batch  184/538 - Train Accuracy: 0.5956, Validation Accuracy: 0.6001, Loss: 0.7850
Epoch   0 Batch  186/538 - Train Accuracy: 0.5938, Validation Accuracy: 0.6048, Loss: 0.8091
Epoch   0 Batch  188/538 - Train Accuracy: 0.5871, Validation Accuracy: 0.6085, Loss: 0.8155
Epoch   0 Batch  190/538 - Train Accuracy: 0.5965, Validation Accuracy: 0.6033, Loss: 0.8261
Epoch   0 Batch  192/538 - Train Accuracy: 0.5917, Validation Accuracy: 0.6048, Loss: 0.7891
Epoch   0 Batch  194/538 - Train Accuracy: 0.5883, Validation Accuracy: 0.6064, Loss: 0.8315
Epoch   0 Batch  196/538 - Train Accuracy: 0.5895, Validation Accuracy: 0.6058, Loss: 0.7674
Epoch   0 Batch  198/538 - Train Accuracy: 0.6276, Validation Accuracy: 0.6085, Loss: 0.7717
Epoch   0 Batch  200/538 - Train Accuracy: 0.6096, Validation Accuracy

Epoch   0 Batch  358/538 - Train Accuracy: 0.7086, Validation Accuracy: 0.7173, Loss: 0.4886
Epoch   0 Batch  360/538 - Train Accuracy: 0.6922, Validation Accuracy: 0.7045, Loss: 0.4996
Epoch   0 Batch  362/538 - Train Accuracy: 0.7117, Validation Accuracy: 0.7063, Loss: 0.4576
Epoch   0 Batch  364/538 - Train Accuracy: 0.7131, Validation Accuracy: 0.7108, Loss: 0.5138
Epoch   0 Batch  366/538 - Train Accuracy: 0.7252, Validation Accuracy: 0.7100, Loss: 0.4931
Epoch   0 Batch  368/538 - Train Accuracy: 0.7676, Validation Accuracy: 0.7008, Loss: 0.4264
Epoch   0 Batch  370/538 - Train Accuracy: 0.6965, Validation Accuracy: 0.6973, Loss: 0.4983
Epoch   0 Batch  372/538 - Train Accuracy: 0.7688, Validation Accuracy: 0.7321, Loss: 0.4739
Epoch   0 Batch  374/538 - Train Accuracy: 0.7197, Validation Accuracy: 0.7141, Loss: 0.4880
Epoch   0 Batch  376/538 - Train Accuracy: 0.7297, Validation Accuracy: 0.7225, Loss: 0.4644
Epoch   0 Batch  378/538 - Train Accuracy: 0.7370, Validation Accuracy

Epoch   0 Batch  536/538 - Train Accuracy: 0.8713, Validation Accuracy: 0.8512, Loss: 0.2383
Epoch   1 Batch    2/538 - Train Accuracy: 0.8717, Validation Accuracy: 0.8562, Loss: 0.2401
Epoch   1 Batch    4/538 - Train Accuracy: 0.8684, Validation Accuracy: 0.8599, Loss: 0.2180
Epoch   1 Batch    6/538 - Train Accuracy: 0.8767, Validation Accuracy: 0.8509, Loss: 0.2017
Epoch   1 Batch    8/538 - Train Accuracy: 0.8582, Validation Accuracy: 0.8455, Loss: 0.2165
Epoch   1 Batch   10/538 - Train Accuracy: 0.8672, Validation Accuracy: 0.8633, Loss: 0.2134
Epoch   1 Batch   12/538 - Train Accuracy: 0.8648, Validation Accuracy: 0.8661, Loss: 0.1996
Epoch   1 Batch   14/538 - Train Accuracy: 0.8881, Validation Accuracy: 0.8395, Loss: 0.1920
Epoch   1 Batch   16/538 - Train Accuracy: 0.8614, Validation Accuracy: 0.8658, Loss: 0.1922
Epoch   1 Batch   18/538 - Train Accuracy: 0.8826, Validation Accuracy: 0.8642, Loss: 0.2035
Epoch   1 Batch   20/538 - Train Accuracy: 0.8830, Validation Accuracy

Epoch   1 Batch  178/538 - Train Accuracy: 0.8981, Validation Accuracy: 0.8888, Loss: 0.1026
Epoch   1 Batch  180/538 - Train Accuracy: 0.9269, Validation Accuracy: 0.8929, Loss: 0.1097
Epoch   1 Batch  182/538 - Train Accuracy: 0.9225, Validation Accuracy: 0.8912, Loss: 0.0995
Epoch   1 Batch  184/538 - Train Accuracy: 0.9183, Validation Accuracy: 0.8947, Loss: 0.1033
Epoch   1 Batch  186/538 - Train Accuracy: 0.9046, Validation Accuracy: 0.9068, Loss: 0.1045
Epoch   1 Batch  188/538 - Train Accuracy: 0.9307, Validation Accuracy: 0.9148, Loss: 0.0930
Epoch   1 Batch  190/538 - Train Accuracy: 0.9089, Validation Accuracy: 0.9022, Loss: 0.1297
Epoch   1 Batch  192/538 - Train Accuracy: 0.9234, Validation Accuracy: 0.8945, Loss: 0.0996
Epoch   1 Batch  194/538 - Train Accuracy: 0.9156, Validation Accuracy: 0.8915, Loss: 0.0990
Epoch   1 Batch  196/538 - Train Accuracy: 0.9195, Validation Accuracy: 0.9045, Loss: 0.0964
Epoch   1 Batch  198/538 - Train Accuracy: 0.9079, Validation Accuracy

...

Epoch   8 Batch  342/538 - Train Accuracy: 0.9762, Validation Accuracy: 0.9739, Loss: 0.0138
Epoch   8 Batch  344/538 - Train Accuracy: 0.9857, Validation Accuracy: 0.9739, Loss: 0.0148
Epoch   8 Batch  346/538 - Train Accuracy: 0.9840, Validation Accuracy: 0.9714, Loss: 0.0141
Epoch   8 Batch  348/538 - Train Accuracy: 0.9834, Validation Accuracy: 0.9712, Loss: 0.0090
Epoch   8 Batch  350/538 - Train Accuracy: 0.9792, Validation Accuracy: 0.9691, Loss: 0.0149
Epoch   8 Batch  352/538 - Train Accuracy: 0.9833, Validation Accuracy: 0.9718, Loss: 0.0235
Epoch   8 Batch  354/538 - Train Accuracy: 0.9848, Validation Accuracy: 0.9737, Loss: 0.0148
Epoch   8 Batch  356/538 - Train Accuracy: 0.9863, Validation Accuracy: 0.9782, Loss: 0.0115
Epoch   8 Batch  358/538 - Train Accuracy: 0.9961, Validation Accuracy: 0.9806, Loss: 0.0087
Epoch   8 Batch  360/538 - Train Accuracy: 0.9855, Validation Accuracy: 0.9766, Loss: 0.0113
Epoch   8 Batch  362/538 - Train Accuracy: 0.9929, Validation Accuracy

Epoch   8 Batch  520/538 - Train Accuracy: 0.9811, Validation Accuracy: 0.9826, Loss: 0.0130
Epoch   8 Batch  522/538 - Train Accuracy: 0.9805, Validation Accuracy: 0.9789, Loss: 0.0118
Epoch   8 Batch  524/538 - Train Accuracy: 0.9850, Validation Accuracy: 0.9771, Loss: 0.0101
Epoch   8 Batch  526/538 - Train Accuracy: 0.9753, Validation Accuracy: 0.9766, Loss: 0.0167
Epoch   8 Batch  528/538 - Train Accuracy: 0.9776, Validation Accuracy: 0.9734, Loss: 0.0162
Epoch   8 Batch  530/538 - Train Accuracy: 0.9783, Validation Accuracy: 0.9716, Loss: 0.0181
Epoch   8 Batch  532/538 - Train Accuracy: 0.9820, Validation Accuracy: 0.9734, Loss: 0.0138
Epoch   8 Batch  534/538 - Train Accuracy: 0.9890, Validation Accuracy: 0.9798, Loss: 0.0104
Epoch   8 Batch  536/538 - Train Accuracy: 0.9840, Validation Accuracy: 0.9798, Loss: 0.0149
Epoch   9 Batch    2/538 - Train Accuracy: 0.9918, Validation Accuracy: 0.9762, Loss: 0.0125
Epoch   9 Batch    4/538 - Train Accuracy: 0.9879, Validation Accuracy

Epoch   9 Batch  162/538 - Train Accuracy: 0.9887, Validation Accuracy: 0.9734, Loss: 0.0115
Epoch   9 Batch  164/538 - Train Accuracy: 0.9852, Validation Accuracy: 0.9680, Loss: 0.0113
Epoch   9 Batch  166/538 - Train Accuracy: 0.9896, Validation Accuracy: 0.9705, Loss: 0.0118
Epoch   9 Batch  168/538 - Train Accuracy: 0.9846, Validation Accuracy: 0.9746, Loss: 0.0130
Epoch   9 Batch  170/538 - Train Accuracy: 0.9818, Validation Accuracy: 0.9743, Loss: 0.0137
Epoch   9 Batch  172/538 - Train Accuracy: 0.9827, Validation Accuracy: 0.9691, Loss: 0.0143
Epoch   9 Batch  174/538 - Train Accuracy: 0.9822, Validation Accuracy: 0.9705, Loss: 0.0133
Epoch   9 Batch  176/538 - Train Accuracy: 0.9842, Validation Accuracy: 0.9716, Loss: 0.0172
Epoch   9 Batch  178/538 - Train Accuracy: 0.9766, Validation Accuracy: 0.9664, Loss: 0.0134
Epoch   9 Batch  180/538 - Train Accuracy: 0.9829, Validation Accuracy: 0.9675, Loss: 0.0129
Epoch   9 Batch  182/538 - Train Accuracy: 0.9732, Validation Accuracy

Epoch   9 Batch  340/538 - Train Accuracy: 0.9861, Validation Accuracy: 0.9698, Loss: 0.0105
Epoch   9 Batch  342/538 - Train Accuracy: 0.9896, Validation Accuracy: 0.9723, Loss: 0.0107
Epoch   9 Batch  344/538 - Train Accuracy: 0.9795, Validation Accuracy: 0.9723, Loss: 0.0145
Epoch   9 Batch  346/538 - Train Accuracy: 0.9862, Validation Accuracy: 0.9695, Loss: 0.0119
Epoch   9 Batch  348/538 - Train Accuracy: 0.9842, Validation Accuracy: 0.9670, Loss: 0.0114
Epoch   9 Batch  350/538 - Train Accuracy: 0.9896, Validation Accuracy: 0.9673, Loss: 0.0137
Epoch   9 Batch  352/538 - Train Accuracy: 0.9818, Validation Accuracy: 0.9675, Loss: 0.0222
Epoch   9 Batch  354/538 - Train Accuracy: 0.9859, Validation Accuracy: 0.9688, Loss: 0.0100
Epoch   9 Batch  356/538 - Train Accuracy: 0.9819, Validation Accuracy: 0.9702, Loss: 0.0110
Epoch   9 Batch  358/538 - Train Accuracy: 0.9914, Validation Accuracy: 0.9725, Loss: 0.0087
Epoch   9 Batch  360/538 - Train Accuracy: 0.9861, Validation Accuracy

Epoch   9 Batch  518/538 - Train Accuracy: 0.9867, Validation Accuracy: 0.9814, Loss: 0.0145
Epoch   9 Batch  520/538 - Train Accuracy: 0.9818, Validation Accuracy: 0.9794, Loss: 0.0144
Epoch   9 Batch  522/538 - Train Accuracy: 0.9881, Validation Accuracy: 0.9764, Loss: 0.0073
Epoch   9 Batch  524/538 - Train Accuracy: 0.9875, Validation Accuracy: 0.9751, Loss: 0.0090
Epoch   9 Batch  526/538 - Train Accuracy: 0.9801, Validation Accuracy: 0.9739, Loss: 0.0124
Epoch   9 Batch  528/538 - Train Accuracy: 0.9883, Validation Accuracy: 0.9789, Loss: 0.0138
Epoch   9 Batch  530/538 - Train Accuracy: 0.9951, Validation Accuracy: 0.9773, Loss: 0.0104
Epoch   9 Batch  532/538 - Train Accuracy: 0.9889, Validation Accuracy: 0.9734, Loss: 0.0070
Epoch   9 Batch  534/538 - Train Accuracy: 0.9868, Validation Accuracy: 0.9737, Loss: 0.0086
Epoch   9 Batch  536/538 - Train Accuracy: 0.9881, Validation Accuracy: 0.9737, Loss: 0.0105
Model Trained and Saved


### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [21]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in sentence.lower().split()]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [22]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [6, 78, 33, 105, 19, 64, 116]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [299, 184, 142, 141, 335, 113, 233, 53, 1]
  French Words: il a vu un vieux camion jaune . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.